In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# Get the API key from the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

# Set the API key in the environment
os.environ["OPENAI_API_KEY"] = api_key

# Check if the API key is loaded properly
if api_key:
    print("API key loaded successfully")
else:
    print("Failed to load API key")

API key loaded successfully


In [2]:
import pandas as pd


# df_monsters = pd.read_csv('monsters.csv', on_bad_lines='skip', sep=";")

# df_monsters.to_csv('df.csv', index=False)


In [3]:
df_monsters = pd.read_csv('df_monsters.csv')
df_monsters

,Name,ChallengeRating,ChallengeXP,ACType,AC,STR,STRMod,DEX,DEXMod,CON,CONMod,INT,INTMod,WIS,WISMod,CHA,CHAMod,HPDice,HP
0,Adult Red Dragon,17,18000,natural armor,19,27,8,10,0,25,7,16,3,13,1,21,5,19d12 + 133,256
1,Stone Golem,10,5900,natural armor,17,22,6,9,-1,20,5,3,-4,11,0,1,-5,17d10 + 85,178
2,Fire Giant,9,5000,plate,18,25,7,9,-1,23,6,10,0,14,2,13,1,13d12 + 78,162
3,Hydra,8,3900,natural armor,15,20,5,12,1,20,5,2,-4,10,0,7,-2,15d12 + 75,172
4,Frost Giant,8,3900,patchwork armor,15,23,6,9,-1,21,5,9,-1,10,0,12,1,12d12 + 60,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Owl,0,10,NaN,11,3,-4,13,1,8,-1,2,-4,12,1,7,-2,1d4 - 1,1
155,Frog,0,0,NaN,11,1,-5,13,1,8,-1,1,-5,8,-1,3,-4,1d4 - 1,1
156,Sea Horse,0,0,NaN,11,1,-5,12,1,8,-1,1,-5,10,0,2,-4,1d4 - 1,1
157,Scorpion,0,10,natural armor,11,2,-4,11,0,8,-1,1,-5,8,-1,2,-4,1d4 - 1,1


In [12]:
# CSV AGENT

from langchain.agents import create_csv_agent

from langchain.chat_models import ChatOpenAI

In [5]:
from platform import python_version

print(python_version())

3.11.2


In [18]:
agent = create_csv_agent(ChatOpenAI(model_name="gpt-4", temperature=0.5), 
                         'df_monsters.csv', 
                         verbose=True)

In [19]:
agent

AgentExecutor(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x0000029971D8F890>, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x0000029971D8F890>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df': '               Name ChallengeRating  ChallengeXP           ACType  AC  STR  \\\n0  Adult Red Dragon              17        18000    natural armor  19   27   \n1       Stone Golem              10         5900    natural armor  17   22   \n2        Fire Giant               9         5000            plate  18   25   \n3             Hydra               8         3900    natural armor  15   20   \n4       Frost Giant               8         3900  patchwork armor  15   23   \n\n   STRMod  DEX  DEXMod  CON  CONMod  INT  INTMod  WIS  WISMod  CHA  CHAMod  \\\n0       8   10       0  

In [20]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [21]:
agent.run("What monsters would be good to appear in a dark forest? Explain why and pick 3 easy and 1 extreme monster")



> Entering new AgentExecutor chain...
Thought: First, I need to filter the dataframe by challenge rating to find easy and extreme monsters. I will consider monsters with challenge ratings between 1 and 3 as easy, and monsters with a challenge rating equal to or greater than 15 as extreme. Then, I will look at the names and features of these monsters to determine which ones would be suitable for a dark forest setting. 

Action: python_repl_ast
Action Input: df[df['ChallengeRating'].between(1, 3)]
Observation: '>=' not supported between instances of 'str' and 'int'
Thought:I received an error message because the challenge rating column is of type string. I need to convert this column to integers before I can perform the filtering.

Action: python_repl_ast
Action Input: df['ChallengeRating'] = df['ChallengeRating'].astype(int)
Observation: invalid literal for int() with base 10: '0,5'
Thought:There is a value '0,5' in the ChallengeRating column, which is not formatted correctly as an in

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 9d198ad662ba77e094b89f3969ec32a3 in your message.).


I have filtered the dataframe to find easy monsters (with challenge ratings between 1 and 3). Now I will do the same for extreme monsters (with challenge ratings equal to or greater than 15).

Action: python_repl_ast
Action Input: df_extreme = df[df['ChallengeRating'] >= 15]
Observation: 
Thought:I have filtered the dataframe to find extreme monsters (with challenge ratings equal to or greater than 15). Now I will examine the features of these monsters to determine which ones would be suitable for a dark forest setting. 

Action: python_repl_ast
Action Input: df_easy.head(10)
Observation:             Name  ChallengeRating  ChallengeXP         ACType  AC  STR  \
27  Killer Whale                3          700  natural armor  12   19   
28      Minotaur                3          700  natural armor  14   18   
29   Winter Wolf                3          700  natural armor  13   18   
30  Ankylosaurus                3          700  natural armor  15   19   
31     Manticore                3 

ValueError: Could not parse LLM output: `Based on the extreme monsters list, I will choose the Adult Red Dragon as the extreme monster for a dark forest setting. Although dragons are not typically associated with forests, an encounter with an Adult Red Dragon could make for an interesting and climactic event in the dark forest setting.

To summarize, the selected monsters for a dark forest encounter are:
1. Winter Wolf (easy)
2. Owlbear (easy)
3. Werewolf (easy)
4. Adult Red Dragon (extreme)`

In [22]:
agent.run("tell me about this data")



> Entering new AgentExecutor chain...
Thought: To tell about the data, I need to check the summary statistics and general information about the dataframe.
Action: python_repl_ast
Action Input: df.describe()
Observation:        ChallengeRating   ChallengeXP          AC         STR      STRMod  \
count       159.000000    159.000000  159.000000  159.000000  159.000000   
mean          1.723270    678.490566   12.622642   13.176101    1.345912   
std           2.557966   1699.135088    1.944415    6.160855    3.058475   
min           0.000000      0.000000    8.000000    1.000000   -5.000000   
25%           0.000000     50.000000   11.000000    8.500000   -1.000000   
50%           0.000000    100.000000   12.000000   14.000000    2.000000   
75%           3.000000    700.000000   14.000000   18.000000    4.000000   
max          17.000000  18000.000000   19.000000   27.000000    8.000000   

              DEX      DEXMod         CON      CONMod         INT      INTMod  \
count  159.0

ValueError: Could not parse LLM output: `I now know the summary statistics of the dataframe.

This dataframe contains information about various creatures such as their name, challenge rating, challenge XP, armor class type, armor class value, and ability scores (STR, DEX, CON, INT, WIS, CHA) along with their corresponding modifiers. Additionally, it has a column for hit points (HP) and hit point dice (HPDice).

Some basic statistics about the dataset are:
- There are 159 creatures in the dataset.
- The average challenge rating is 1.72 with a standard deviation of 2.56. The maximum challenge rating is 17.
- The average armor class is 12.62 with a standard deviation of 1.94.
- The creatures have an average hit point value of 40.74 with a standard deviation of 43.79. The maximum hit point value is 256.

Note that these statistics apply to the numeric columns in the dataframe. For categorical columns such as `ACType`, a different type of analysis would be required to summarize the data.`

In [23]:
agent.run("give me 5 creatures with different features that may be interesting in a fantasy setting")



> Entering new AgentExecutor chain...
Thought: I need to select 5 random rows from the dataframe
Action: python_repl_ast
Action Input: df.sample(5)
Observation:               Name  ChallengeRating  ChallengeXP              ACType  AC  STR  \
32         Owlbear                3          700       natural armor  13   20   
94       Hobgoblin                0          100  chain mail, shield  18   13   
140         Jackal                0           10                 NaN  12    8   
88        Warhorse                0          100                 NaN  11   18   
45   Awakened Tree                2          450       natural armor  13   19   

     STRMod  DEX  DEXMod  CON  CONMod  INT  INTMod  WIS  WISMod  CHA  CHAMod  \
32        5   12       1   17       3    3      -4   12       1    7      -2   
94        1   12       1   12       1   10       0   10       0    9      -1   
140      -1   15       2   11       0    3      -4   12       1    6      -2   
88        4   12       1   13 

'Here are 5 interesting creatures:\n1. Owlbear - A mix between an owl and a bear with a natural armor.\n2. Hobgoblin - A goblin-like creature wearing chain mail armor and carrying a shield.\n3. Jackal - A small, dog-like creature.\n4. Warhorse - A strong, powerful horse trained for battle.\n5. Awakened Tree - A sentient tree with natural armor.'

In [24]:
agent.run("give me some info on adult red dragon that I can use for storytelling and combat. Give me also an descriptive explanation of it that would fit. Give me something angry it could say as players meet it for the first time")



> Entering new AgentExecutor chain...
Thought: First, let's check the information about Adult Red Dragon that we have in our dataframe. We can filter the dataframe to get the information for the Adult Red Dragon.
Action: python_repl_ast
Action Input: df[df['Name'] == 'Adult Red Dragon']
Observation:                Name  ChallengeRating  ChallengeXP         ACType  AC  STR  \
0  Adult Red Dragon               17        18000  natural armor  19   27   

   STRMod  DEX  DEXMod  CON  CONMod  INT  INTMod  WIS  WISMod  CHA  CHAMod  \
0       8   10       0   25       7   16       3   13       1   21       5   

        HPDice   HP  
0  19d12 + 133  256  
Thought:

ValueError: Could not parse LLM output: `Now we have the stats and information for the Adult Red Dragon. Let's prepare a descriptive explanation of it and what it might say.

The Adult Red Dragon is a fearsome and mighty creature, with a challenge rating of 17 and 18,000 experience points. It is protected by thick, natural armor giving it an Armor Class of 19. The dragon's immense strength grants it a +8 strength modifier, while its constitution lends it a +7 modifier. With an above-average intelligence and charisma, the dragon is capable of outsmarting enemies and using its force of personality to its advantage.

The dragon's body is covered in red, flame-like scales that seem to shimmer and dance like the fires of the inferno. Its massive wings cast ominous shadows as it soars overhead, while its enormous, curved horns and frightening golden eyes pierce the very souls of those who behold it. The Adult Red Dragon's health points are calculated with 19d12 + 133, with a total of 256 hea